# Figure of station profile model evaluation


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import netCDF4 as nc
import cmocean
import pandas as pd 
from matplotlib.patches import ConnectionPatch
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')
import pickle
from datetime import date
from datetime import datetime
import xarray as xr
import glob
import gsw

%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '/ocean/brogalla/GEOTRACES/analysis-brogalla/modules')
import ariane as ar
import seaborn as sns

###### Parameters:

In [16]:
# domain dimensions:
imin, imax = 1480, 2180
jmin, jmax = 160, 800

# results to plot:
folder_ref_2015      = f'/data/brogalla/run_storage/Mn-tuning-202107/ANHA12_ref-2015_euphotic/'
folder_ref_2009      = f'/data/brogalla/run_storage/Mn-tuning-202107/ANHA12_ref-2009_euphotic/'

# colours:
c_continent    = '#ce9169'
c_glacier      = '#36ab92'
c_other        = '#a6a6a6'
land_color     = "#8b7765"
profile_colors = ['#e54141', '#33c7ff']

##### Load files:

In [17]:
def calculate_average(folder_ref_2009, folder_ref_2015):
    # Calculate average over data files during cruise period:
    # 2009 GEOTRACES: 29 August - 8 September
    files_2009      = np.sort(glob.glob(f'{folder_ref_2009}ANHA12_EXH006_5d_20090101_20091231*'))
    start_date_2009 = datetime.strptime('2009-08-29', '%Y-%m-%d')
    end_date_2009   = datetime.strptime('2009-09-08', '%Y-%m-%d')

    file_list_2009 = []
    for file in files_2009:
        if (datetime.strptime(file.split('/')[-1][42:50],'%Y%m%d') >= start_date_2009) & \
                (datetime.strptime(file.split('/')[-1][51:59],'%Y%m%d') < end_date_2009):
            file_list_2009.append(file)
    dmn_2009 = np.zeros((50,700,640))
    days = 0
    for file in file_list_2009:
        ds = xr.open_dataset(f"{file}")
        dmn_2009 = dmn_2009 + ds['dissolmn'].values[0,:,:,:]

    mn_dis_ref_2009 = dmn_2009 / len(file_list_2009)

    # 2015 GEOTRACES: 9 August - 15 September
    files_2015      = np.sort(glob.glob(f'{folder_ref_2015}ANHA12_EXH006_5d_20150101_20151231*'))
    start_date_2015 = datetime.strptime(f'2015-08-09', '%Y-%m-%d')
    end_date_2015   = datetime.strptime(f'2015-09-15', '%Y-%m-%d')

    file_list_2015 = []
    for file in files_2015:
        if (datetime.strptime(file.split('/')[-1][42:50],'%Y%m%d') >= start_date_2015) & \
                    (datetime.strptime(file.split('/')[-1][51:59],'%Y%m%d') < end_date_2015):
            file_list_2015.append(file)

    dmn_2015 = np.zeros((50,700,640))
    days = 0
    for file in file_list_2015:
        ds = xr.open_dataset(f"{file}")
        dmn_2015 = dmn_2015 + ds['dissolmn'].values[0,:,:,:]

    mn_dis_ref_2015 = dmn_2015 / len(file_list_2015)
    
    return mn_dis_ref_2009, mn_dis_ref_2015

In [18]:
mn_ref_2009, mn_ref_2015           = calculate_average(folder_ref_2009, folder_ref_2015)

Model results

In [19]:
model_ref_2015 = nc.Dataset(f'{folder_ref_2015}ANHA12_EXH006_2015_monthly.nc')
lon_model    = np.array(model_ref_2015.variables['nav_lon'])
lat_model    = np.array(model_ref_2015.variables['nav_lat'])
depths_model = np.array(model_ref_2015.variables['deptht'])

Meshes

In [20]:
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_lon   = np.array(mesh.variables['nav_lon'])
mesh_lat   = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['hdept'][0])
tmask    =  np.array(mesh.variables['tmask'])[0,:,imin:imax,jmin:jmax]
Z_masked = np.ma.masked_where((tmask > 0.1), tmask) 
e3t = np.array(mesh.variables['e3t_0'])

Observations

In [21]:
# Concentration of dMn in the Beaufort Sea in 2009.
### Thesis doesn't have a list of lats and lons of station locations --- Jeffrey Charter's thesis does (2012)
IPY = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Nari_Sim_dMn_data.csv')
IPY_names_full = IPY['Station name'].astype('string').values
IPY_depths     = IPY['Depth [m]'].astype('float').values
IPY_dMn_nmolkg = IPY['DMn [ nmol/kg]'].astype('float').values
IPY_lons_full  = -1*IPY['Longitude'].astype('float').values
IPY_lats_full  = IPY['Latitude'].astype('float').values

# Unique entries (maintaining order)
IPY_lons  = np.array(list(dict.fromkeys(IPY_lons_full)))
IPY_lats  = np.array(list(dict.fromkeys(IPY_lats_full)))
IPY_names = np.array(list(dict.fromkeys(IPY_names_full)))

# Convert nmol / kg to nmol / L
IPY_density             = IPY['Density'].astype('float').values - 1000 # kg / L
IPY_depths_cruise_sheet = IPY['Depth [m] cruise sheet'].astype('float').values
IPY_dMn_nmolL = IPY_dMn_nmolkg * (IPY_density+1000)/1000

In [22]:
# Concentration of dMn in 2015
# Manuel = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Fe&Mn_Integrated_Data_UBC-UVic_4-17-19_editedBR.csv')
Manuel = pd.read_csv('/ocean/brogalla/GEOTRACES/data/DMn_nmolL-1_BR-edited.csv')
Manuel_station_names = Manuel['Station'].astype('string')
Manuel_depths        = Manuel['Depth'].astype('float')
Manuel_dMn_nmolkg    = Manuel['nmol kg-1'].astype('float').values
Manuel_dMn_nmolL     = Manuel['nmol L-1'].astype('float').values
Manuel_salinity      = Manuel['Salinity'].astype('float').values

In [23]:
GEOTRACES = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Pb-paper-data.csv')
stn_names = GEOTRACES['Station names'].dropna().astype('str')
lons      = GEOTRACES['station lon'].astype('float').dropna().values
lats      = GEOTRACES['Station lat'].astype('float').dropna().values

##### Define functions

In [24]:
def find_index_ANHA12_sub(transect_lons, transect_lats, ANHA12sub_lon=lon_model, ANHA12sub_lat=lat_model):
    # input:   ANHA12sub_lon, ANHA12sub_lat --- 570x600 coordinates
    #          transect_lons, transect_lats --- list of lat and lon values to include in the transect
    # output:  i, j                         --- lists of coordinates associated with these points
    
    transect_i = np.array([])
    transect_j = np.array([])
    
    if isinstance(transect_lons,float):
        i, j = ar.find_closest_model_point(transect_lons, transect_lats, ANHA12sub_lon, ANHA12sub_lat)
        transect_i = int(i)
        transect_j = int(j)
    else:
        for k in range(0,len(transect_lons)):
            i, j = ar.find_closest_model_point(transect_lons[k], transect_lats[k], ANHA12sub_lon, ANHA12sub_lat)
            transect_i = np.append(transect_i, int(i))
            transect_j = np.append(transect_j, int(j))
    
    return transect_i, transect_j

In [44]:
def station_info_2015(name):

    dMn_obs   = Manuel_dMn_nmolL[np.array(Manuel_station_names==name).astype('bool')]
    depth_obs = Manuel_depths[np.array(Manuel_station_names==name).astype('bool')].values
    obs_e3t   = np.diff(depth_obs, prepend=0) # size of vertical boxes

    lon_station = lons[stn_names==name]
    lat_station = lats[stn_names==name]

    model_stni, model_stnj = find_index_ANHA12_sub(lon_station, lat_station)
    model_stni = int(model_stni); model_stnj = int(model_stnj);
    
    e3t_stn = np.ma.masked_where((tmask[:,model_stni-1,model_stnj-1] < 0.1), e3t[0,:,model_stni+imin-1,model_stnj+jmin-1])

    mn_ref = np.ma.masked_where((tmask[:,model_stni-1,model_stnj-1] < 0.1), mn_ref_2015[:, model_stni, model_stnj]*10**9)
    
    return obs_e3t, dMn_obs, e3t_stn, mn_ref

def station_info_2009(name):

    dMn_obs   = IPY_dMn_nmolL[np.array(IPY_names_full==name).astype('bool')]
    depth_obs = IPY_depths_cruise_sheet[np.array(IPY_names_full==name).astype('bool')]
    obs_e3t   = np.diff(depth_obs, prepend=0) # size of vertical boxes

    lon_station = IPY_lons[IPY_names==name]
    lat_station = IPY_lats[IPY_names==name]

    model_stni, model_stnj = find_index_ANHA12_sub(lon_station, lat_station)
    model_stni = int(model_stni); model_stnj = int(model_stnj);
    
    e3t_stn = np.ma.masked_where((tmask[:,model_stni-1,model_stnj-1] < 0.1), e3t[0,:,model_stni+imin-1,model_stnj+jmin-1])

    mn_ref = np.ma.masked_where((tmask[:,model_stni-1,model_stnj-1] < 0.1), mn_ref_2009[:, model_stni, model_stnj]*10**9)
    
    return obs_e3t, dMn_obs, e3t_stn, mn_ref

In [101]:
def depth_integrate(name, year=''):
    if year=='2015':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2015(name)
    elif year=='2009':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2009(name)
    else:
        print('Not a year with Mn evaluation data')
    
    # depth-integrated Mn content:
    obs_totaldMn_stn   = obs_e3t_stn*obs_dMn_stn
    model_totaldMn_stn = model_e3t_stn*model_dMn_stn
    
    # average Mn in water column:
    obs_ave_dMn   = sum(obs_totaldMn_stn)/sum(obs_e3t_stn)
    model_ave_dMn = np.ma.sum(model_totaldMn_stn)/np.ma.sum(model_e3t_stn)
    
    return obs_ave_dMn, model_ave_dMn

def depth_integrate_subsurface(name, year=''):
    if year=='2015':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2015(name)
    elif year=='2009':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2009(name)
    else:
        print('Not a year with Mn evaluation data')
    
    subsurface_depth = 52
    obs_depth   = np.cumsum(obs_e3t_stn) 
    model_depth = np.cumsum(model_e3t_stn)
    
    # depth-integrated Mn content:
    obs_totaldMn_stn   = sum(obs_e3t_stn[obs_depth < subsurface_depth]*obs_dMn_stn[obs_depth < subsurface_depth])
    model_totaldMn_stn = np.ma.sum(model_e3t_stn[model_depth < subsurface_depth]*model_dMn_stn[model_depth < subsurface_depth])
    
    # average Mn in water column:
    obs_ave_dMn   = obs_totaldMn_stn/np.amax(obs_depth[obs_depth < subsurface_depth])
    model_ave_dMn = model_totaldMn_stn/np.amax(model_depth[model_depth < subsurface_depth])
    
    return obs_ave_dMn, model_ave_dMn

def total_dMn_subsurface(name, year=''):
    if year=='2015':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2015(name)
    elif year=='2009':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2009(name)
    else:
        print('Not a year with Mn evaluation data')
    
    subsurface_depth = 52
    obs_depth   = np.cumsum(obs_e3t_stn) 
    model_depth = np.cumsum(model_e3t_stn)
    
    # depth-integrated Mn content:
    obs_totaldMn_stn   = sum(obs_e3t_stn[obs_depth < subsurface_depth]*obs_dMn_stn[obs_depth < subsurface_depth])
    model_totaldMn_stn = np.ma.sum(model_e3t_stn[model_depth < subsurface_depth]*model_dMn_stn[model_depth < subsurface_depth])
    
    print(f'Total dMn at {name}: obs (upper {np.amax(obs_depth[obs_depth < subsurface_depth]):.1f}m): '+
          f'{obs_totaldMn_stn:.2f}, model (upper {np.amax(model_depth[model_depth < subsurface_depth]):.1f}m): {model_totaldMn_stn:.2f}')
    
    return obs_totaldMn_stn, model_totaldMn_stn

In [ ]:
def depth_integrate_subsurface(name, year=''):

    if year=='2015':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2015(name)
    elif year=='2009':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2009(name)
    else:
        print('Not a year with Mn evaluation data')
        
    obs_depths   = np.cumsum(obs_e3t_stn)
    model_depths = np.ma.cumsum(model_e3t_stn)
        
    # define sub-surface depth
    subsurface_depth = 50
        
    # depth-integrated Mn content of the sub-surface:
    obs_totaldMn_stn   = obs_e3t_stn[obs_depths < subsurface_depth]*obs_dMn_stn[obs_depths < subsurface_depth]
    model_totaldMn_stn = model_e3t_stn[model_depths < subsurface_depth]*model_dMn_stn[model_depths < subsurface_depth]
    print(f'Total dMn in upper 50 m, observations: {sum(obs_totaldMn_stn):.2f}, model: {np.ma.sum(model_totaldMn_stn):.2f}')
    
    # average Mn in sub-surface:
    obs_ave_dMn   = sum(obs_totaldMn_stn)/sum(obs_e3t_stn[obs_depths < subsurface_depth])
    model_ave_dMn = np.ma.sum(model_totaldMn_stn)/np.ma.sum(model_e3t_stn[model_depths < subsurface_depth])
    
    return obs_ave_dMn, model_ave_dMn

#### Calculate:

In [28]:
obs_e3t_CAA9, obs_dMn_CAA9, model_e3t_CAA9, model_dMn_CAA9 = station_info_2015('CAA9')

In [29]:
obs_ave_dMn_CAA9, model_ave_dMn_CAA9 = depth_integrate('CAA9', year='2015')
print(obs_ave_dMn_CAA9, model_ave_dMn_CAA9)

4.335330757407887 7.456815753722486


In [55]:
obs_ave_dMn_CAA4, model_ave_dMn_CAA4 = depth_integrate_subsurface('CAA4', year='2015')
obs_ave_dMn_CAA5, model_ave_dMn_CAA5 = depth_integrate_subsurface('CAA5', year='2015')
obs_ave_dMn_CAA6, model_ave_dMn_CAA6 = depth_integrate_subsurface('CAA6', year='2015')
obs_ave_dMn_CAA9, model_ave_dMn_CAA9 = depth_integrate_subsurface('CAA9', year='2015')

In [102]:
obs_total_dMn_CAA4, model_total_dMn_CAA4 = total_dMn_subsurface('CAA4', year='2015')
obs_total_dMn_CAA5, model_total_dMn_CAA5 = total_dMn_subsurface('CAA5', year='2015')
obs_total_dMn_CAA6, model_total_dMn_CAA6 = total_dMn_subsurface('CAA6', year='2015')
obs_total_dMn_CAA9, model_total_dMn_CAA9 = total_dMn_subsurface('CAA9', year='2015')

Total dMn at CAA4: obs (upper 47.0m): 268.80, model (upper 51.3m): 192.67
Total dMn at CAA5: obs (upper 45.1m): 242.26, model (upper 51.3m): 155.94
Total dMn at CAA6: obs (upper 32.7m): 151.86, model (upper 51.3m): 186.34
Total dMn at CAA9: obs (upper 49.8m): 292.06, model (upper 51.3m): 205.26


In [103]:
stn_list = ['CAA1', 'CAA2', 'CAA3', 'CAA4', 'CAA5', 'CAA6', 'CAA7', 'CAA8', 'CAA9']
combined_difference = 0; combined_observed=0; combined_modelled=0;
print('Subsurface averaged Mn concentration:')
for stn in stn_list:
    obs_ave_dMn, model_ave_dMn = depth_integrate_subsurface(stn, year='2015')
    print(f'{stn}: observed {obs_ave_dMn:.2f}, modelled {model_ave_dMn:.2f}, difference {model_ave_dMn-obs_ave_dMn:.2f}' )
    combined_difference += model_ave_dMn - obs_ave_dMn
    combined_observed += obs_ave_dMn
    combined_modelled += model_ave_dMn

print(f'Average observed subsurface Mn concentration: {combined_observed/len(stn_list):.2f}')
print(f'Average modelled subsurface Mn concentration: {combined_modelled/len(stn_list):.2f}')
print(f'Average difference between modelled and observed subsurface Mn concentration: {combined_difference/len(stn_list):.2f}')

Subsurface averaged Mn concentration:
CAA1: observed 5.22, modelled 5.79, difference 0.57
CAA2: observed 3.56, modelled 5.03, difference 1.47
CAA3: observed 4.95, modelled 3.42, difference -1.52
CAA4: observed 5.72, modelled 3.75, difference -1.97
CAA5: observed 5.37, modelled 3.04, difference -2.33
CAA6: observed 4.65, modelled 3.63, difference -1.02
CAA7: observed 4.39, modelled 3.68, difference -0.71
CAA8: observed 4.08, modelled 3.14, difference -0.94
CAA9: observed 5.87, modelled 4.00, difference -1.87
Average observed subsurface Mn concentration: 4.87
Average modelled subsurface Mn concentration: 3.94
Average difference between modelled and observed subsurface Mn concentration: -0.92


In [21]:
stn_list = ['CB1', 'CB2', 'CB3', 'CB4']
combined_difference = 0; combined_observed=0; combined_modelled=0;
print('Subsurface averaged Mn concentration:')
for stn in stn_list:
    obs_ave_dMn, model_ave_dMn = depth_integrate_subsurface(stn, year='2015')
    print(f'{stn}: observed {obs_ave_dMn:.2f}, modelled {model_ave_dMn:.2f}, difference {model_ave_dMn-obs_ave_dMn:.2f}' )
    combined_difference += model_ave_dMn - obs_ave_dMn
    combined_observed += obs_ave_dMn
    combined_modelled += model_ave_dMn

print(f'Average observed subsurface Mn concentration: {combined_observed/len(stn_list):.2f}')
print(f'Average modelled subsurface Mn concentration: {combined_modelled/len(stn_list):.2f}')
print(f'Average difference between modelled and observed subsurface Mn concentration: {combined_difference/len(stn_list):.2f}')

Subsurface averaged Mn concentration:
CB1: observed 6.20, modelled 2.49, difference -3.70
CB2: observed 3.54, modelled 7.19, difference 3.65
CB3: observed 4.58, modelled 3.22, difference -1.36
CB4: observed 6.04, modelled 1.44, difference -4.61
Average observed subsurface Mn concentration: 5.09
Average modelled subsurface Mn concentration: 3.58
Average difference between modelled and observed subsurface Mn concentration: -1.51


In [22]:
stn_list = ['L1','L1.1','L1.5','L2','L3','S4']
combined_difference = 0; combined_observed=0; combined_modelled=0;
print('Subsurface averaged Mn concentration:')
for stn in stn_list:
    obs_ave_dMn, model_ave_dMn = depth_integrate_subsurface(stn, year='2009')
    print(f'{stn}: observed {obs_ave_dMn:.2f}, modelled {model_ave_dMn:.2f}, difference {model_ave_dMn-obs_ave_dMn:.2f}' )
    combined_difference += model_ave_dMn - obs_ave_dMn
    combined_observed += obs_ave_dMn
    combined_modelled += model_ave_dMn

print(f'Average observed subsurface Mn concentration: {combined_observed/len(stn_list):.2f}')
print(f'Average modelled subsurface Mn concentration: {combined_modelled/len(stn_list):.2f}')
print(f'Average difference between modelled and observed subsurface Mn concentration: {combined_difference/len(stn_list):.2f}')

Subsurface averaged Mn concentration:
L1: observed 3.88, modelled 5.12, difference 1.24
L1.1: observed 4.10, modelled 3.34, difference -0.76
L1.5: observed 4.95, modelled 4.55, difference -0.40
L2: observed 4.83, modelled 5.45, difference 0.61
L3: observed 3.10, modelled 3.46, difference 0.36
S4: observed 4.10, modelled 2.35, difference -1.75
Average observed subsurface Mn concentration: 4.16
Average modelled subsurface Mn concentration: 4.05
Average difference between modelled and observed subsurface Mn concentration: -0.11


In [114]:
stn_list = ['CAA1', 'CAA2', 'CAA3', 'CAA4', 'CAA5', 'CAA6', 'CAA7', 'CAA8', 'CAA9']
combined_difference = 0
print('Full water column averaged Mn concentration:')
for stn in stn_list:
    obs_ave_dMn, model_ave_dMn = depth_integrate(stn)
    print(f'{stn}: observed {obs_ave_dMn:.2f}, modelled {model_ave_dMn:.2f}, difference {model_ave_dMn-obs_ave_dMn:.2f}' )
    combined_difference += model_ave_dMn - obs_ave_dMn
    
print(f'Average difference between modelled and observed full water column Mn concentration: {combined_difference/len(stn_list):.2f}')

Full water column averaged Mn concentration:
CAA1: observed 3.58, modelled 2.15, difference -1.43
CAA2: observed 4.08, modelled 2.27, difference -1.81
CAA3: observed 2.01, modelled 2.19, difference 0.18
CAA4: observed 3.93, modelled 2.28, difference -1.65
CAA5: observed 4.84, modelled 3.48, difference -1.36
CAA6: observed 5.43, modelled 3.36, difference -2.07
CAA7: observed 4.06, modelled 2.23, difference -1.83
CAA8: observed 1.06, modelled 1.18, difference 0.12
CAA9: observed 4.34, modelled 7.46, difference 3.12
Average difference between modelled and observed full water column Mn concentration: -0.75
